# inverse medium scattering problem
This is an example for the `regpy.solvers.nonlinear.irgnm.IrgnmCG` solver for the example of __medium scattering__

In medium scattering we try to determine a perturbation $f$ the refractive index $1+f$ of a medium from measurements of far ﬁeld patterns $u_{\infty}$ of scattered time-harmonic acoustic waves $u - u_{inc}$ in this medium. The total field $u$ satisfies 

$$
 \Delta u + k^2 (1+f) u = 0 \qquad \text{in } \mathbb{R}^2
$$

The __Iteratively Regularized Gauss-Newton Method (IRGNM)__ minimizes in each iteration 
$$
  f_{n+1}  = f_n + \argmin_{h} \Vert F(f_{n}) + F'[f_n] h - u_{\infty}^{obs}\Vert^{2} + \alpha_{n}  \Vert f_{n} + h - f_0\Vert^{2}
$$
where $F$ is a Frechet-differentiable operator, using `regpy.solvers.linear.tikhonov.TikhonovCG`.
$\alpha_n$ is a decreasing geometric sequence of regularization parameters.

In [ ]:
from examples.medium_scattering.mediumscattering import MediumScatteringFixed
from regpy.operators import CoordinateProjection
from regpy.hilbert import L2, HmDomain
from regpy.solvers import RegularizationSetting
from regpy.solvers.nonlinear.irgnm import IrgnmCG
import regpy.stoprules as rules
import regpy.util as util

import numpy as np
import logging

import matplotlib.pyplot as plt
import matplotlib.colorbar as cbar


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(name)-20s :: %(message)s'
)

In [ ]:
# building the forwrd  operator
radius = 1
scattering = MediumScatteringFixed(
    gridshape=(64, 64),
    radius=radius,
    wave_number=1,
    inc_directions=util.linspace_circle(16),
    farfield_directions=util.linspace_circle(16),
)

projection = CoordinateProjection(
    scattering.domain,
    scattering.support
)
embedding = projection.adjoint

op = scattering * embedding



In [ ]:
#creating data
contrast = scattering.domain.zeros()
r = np.linalg.norm(scattering.domain.coords, axis=0)
contrast[r < radius] = np.exp(-1/(radius - r[r < radius]**2))



exact_solution = projection(contrast)
exact_data = op(exact_solution)
# create and add noise
noise = 0.001 * op.codomain.randn()
data = exact_data + noise
init = op.domain.zeros()


#plotting 
fig,axs = plt.subplots(2,3,figsize=(8,4))
fig.tight_layout()
ax = axs[0,0]
im = ax.imshow(contrast.real)
fig.colorbar(im,ax=ax)
ax.set_ylabel('real')
ax.set_title('exact solution')
ax = axs[0,1]
im = ax.imshow(exact_data.real)
fig.colorbar(im,ax=ax)
ax.set_title('exact data')
ax = axs[0,2]
im = ax.imshow(data.real)
fig.colorbar(im,ax=ax)
ax.set_title('noisy data')
ax = axs[1,0]
ax.set_ylabel('imagenary')
im = ax.imshow(contrast.imag)
fig.colorbar(im,ax=ax)
ax.set_title('exact solution')
ax = axs[1,1]
im = ax.imshow(exact_data.imag)
fig.colorbar(im,ax=ax)
ax.set_title('exact data')
ax = axs[1,2]
im = ax.imshow(data.imag)
fig.colorbar(im,ax=ax)
ax.set_title('noisy data')
plt.show();

In [ ]:
#create setting
myh_domain = HmDomain(scattering.domain,scattering.support,dtype=complex,index=2)
setting = RegularizationSetting(
    op=op,
    # Define Sobolev norm on support via embedding
    penalty = myh_domain, 
    data_fid=L2
)


In [ ]:
#set up solver
solver = IrgnmCG(
    setting, data,
    regpar=0.0001, regpar_step=0.8,
    init=init,
    cg_pars=dict(
        tol=1e-8,
        reltolx=1e-8,
        reltoly=1e-8
    )
)
#set up stopping creiteria
stoprule = (
    rules.CountIterations(100) +
    rules.Discrepancy(
        setting.h_codomain.norm, data,
        noiselevel=setting.h_codomain.norm(noise),
        tau=1.1
    )
)
reco, reco_data = solver.run(stoprule)



In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, constrained_layout=True,figsize=(8,4))
bars = np.vectorize(lambda ax: cbar.make_axes(ax)[0], otypes=[object])(axes)

axes[0, 0].set_title('exact contrast')
axes[1, 0].set_title('exact data')
axes[0, 1].set_title('reco contrast')
axes[1, 1].set_title('reco data')
axes[0, 2].set_title('difference')

def show(i, j, x):
    im = axes[i, j].imshow(x)
    bars[i, j].clear()
    fig.colorbar(im, cax=bars[i, j])

show(0, 0, np.abs(contrast))
show(1, 0, np.abs(exact_data))
solution = embedding(reco)
show(0, 1, np.abs(solution))
show(1, 1, np.abs(reco_data))
show(0, 2, np.abs(solution - contrast))
show(1, 2, np.abs(exact_data - reco_data))
plt.show();
